Реализуйте алгоритм SAC для среды lunar lander

In [1]:
!pip install swig
!pip install "gymnasium[box2d]"

   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   -------------------- ------------------- 1.3/2.6 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for box2d-py: filename=box2d_py-2.3.5-cp311-cp311-win_amd64.whl size=439855 sha256=143c383ff9538483d8d65e45d883483e0baaec3e315803f4f1ac97dc60ac8913
  Stored in directory: c:\users\digital razor\appdata\local\pip\cache\wheels\ab\f1\0c\d56f4a2bdd12bae0a0693ec33f2f0daadb5eb9753c78fa5308
Successfully built box2d-py


In [57]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
import random
from torch.distributions import Normal

In [58]:
GAMMA = 0.99
TAU = 0.005
ALPHA = 0.2
ACTOR_LR = 3e-4
CRITIC_LR = 3e-4
REPLAY_SIZE = 100000
BATCH_SIZE = 256
START_STEPS = 10000
TOTAL_STEPS = 200000
UPDATE_AFTER = 1000
UPDATE_EVERY = 50

In [67]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [68]:
class Actor(nn.Module):
    def __init__(self, obs_dim, act_dim, act_limit):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_dim, 256), nn.ReLU(),
            nn.Linear(256, 256), nn.ReLU(),
        )
        self.mu_layer = nn.Linear(256, act_dim)
        self.log_std_layer = nn.Linear(256, act_dim)
        self.act_limit = act_limit

    def forward(self, obs):
        x = F.relu(self.net(obs))
        mean, std = self.mu_layer(x),  torch.clamp(self.log_std_layer(x), -20, 2).exp()
        normal = torch.distributions.Normal(mean, std)

        x_t = normal.rsample()
        y_t = torch.tanh(x_t)
        action = y_t * (self.act_limit[1] - self.act_limit[0]) / 2.0 + (self.act_limit[0] + self.act_limit[1]) / 2.0

        log_prob = normal.log_prob(x_t)
        log_prob -= torch.log((1 - y_t.pow(2)) + 1e-6)
        log_prob = log_prob.sum(1, keepdim=True)

        return action, log_prob

    def get_action(self, obs, deterministic=False):
        if not isinstance(obs, torch.Tensor):
            obs = torch.tensor(obs, dtype=torch.float32, device=device)

        x = self.net(obs)
        mean = self.mu_layer(x)
        log_std = torch.clamp(self.log_std_layer(x), -20, 2)
        std = log_std.exp()

        normal = torch.distributions.Normal(mean, std)

        if deterministic:
            x_t = mean
        else:
            x_t = normal.rsample()

        y_t = torch.tanh(x_t)

        action = y_t * (self.act_limit[1] - self.act_limit[0]) / 2.0 + (self.act_limit[0] + self.act_limit[1]) / 2.0

        return action.squeeze(0).detach().cpu().numpy()

In [69]:
class Critic(nn.Module):
    def __init__(self, obs_dim, act_dim):
        super().__init__()
        self.q1 = nn.Sequential(
            nn.Linear(obs_dim + act_dim, 256), nn.ReLU(),
            nn.Linear(256, 256), nn.ReLU(),
            nn.Linear(256, 1)
        )
        self.q2 = nn.Sequential(
            nn.Linear(obs_dim + act_dim, 256), nn.ReLU(),
            nn.Linear(256, 256), nn.ReLU(),
            nn.Linear(256, 1)
        )

    def forward(self, obs, act):
        if isinstance(act, tuple):
            act = act[0]
        x = torch.cat([obs, act], dim=-1)
        return self.q1(x), self.q2(x)

In [70]:
class ReplayBuffer:
    def __init__(self, size):
        self.buffer = deque(maxlen=size)

    def add(self, *args):
        self.buffer.append(tuple(args))

    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = map(np.array, zip(*batch))
        return (
            torch.tensor(states, dtype=torch.float32).to(device),
            torch.tensor(actions, dtype=torch.float32).to(device),
            torch.tensor(rewards, dtype=torch.float32).unsqueeze(1).to(device),
            torch.tensor(next_states, dtype=torch.float32).to(device),
            torch.tensor(dones, dtype=torch.float32).unsqueeze(1).to(device)
        )

    def __len__(self):
        return len(self.buffer)

In [71]:
env = gym.make("LunarLanderContinuous-v3")
obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]
action_low, action_high = float(env.action_space.low[0]), float(env.action_space.high[0])
act_limit = [action_low, action_high]

actor = Actor(obs_dim, act_dim, act_limit)
critic = Critic(obs_dim, act_dim)

actor_target = Actor(obs_dim, act_dim, act_limit) 
critic_target = Critic(obs_dim, act_dim)

actor_target.load_state_dict(actor.state_dict())
critic_target.load_state_dict(critic.state_dict())

actor_opt = optim.Adam(actor.parameters(), lr=ACTOR_LR)
critic_opt = optim.Adam(critic.parameters(), lr=CRITIC_LR)

replay = ReplayBuffer(REPLAY_SIZE)

obs, _ = env.reset()
episode_return, episode_len = 0, 0

In [72]:
def update(actor, critic, actor_target, critic_target, replay_buffer, actor_opt, critic_opt, batch_size, gamma, tau):
    if len(replay_buffer) < batch_size:
        return

    states, actions, rewards, next_states, dones = replay_buffer.sample(batch_size)

    # ------------------------ Обновление Critic ------------------------
    with torch.no_grad():
        next_actions = actor_target(next_states)
        if isinstance(next_actions, tuple):
            next_actions = next_actions[0]

        target_q1, target_q2 = critic_target(next_states, next_actions)
        target_q = torch.min(target_q1, target_q2)

        target_q = rewards + gamma * (1 - dones) * target_q.squeeze()

    current_q1, current_q2 = critic(states, actions)

    critic_loss = nn.MSELoss()(current_q1.squeeze(), target_q) + nn.MSELoss()(current_q2.squeeze(), target_q)

    critic_opt.zero_grad()
    critic_loss.backward()
    critic_opt.step()

    # ------------------------ Обновление Actor ------------------------
    actor_loss = -critic(states, actor(states))[0].mean()

    actor_opt.zero_grad()
    actor_loss.backward()
    actor_opt.step()

    # ------------------------ Мягкое обновление целевых сетей ------------------------
    for param, target_param in zip(critic.parameters(), critic_target.parameters()):
        target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

    for param, target_param in zip(actor.parameters(), actor_target.parameters()):
        target_param.data.copy_(tau * param.data + (1 - tau) * target_param.data)

In [73]:
actor.to(device)
critic.to(device)
actor_target.to(device)
critic_target.to(device)

Critic(
  (q1): Sequential(
    (0): Linear(in_features=10, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=1, bias=True)
  )
  (q2): Sequential(
    (0): Linear(in_features=10, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=1, bias=True)
  )
)

In [ ]:
for step in range(TOTAL_STEPS):
    if step < START_STEPS:
        act = env.action_space.sample()
    else:
        with torch.no_grad():
            obs_t = torch.tensor(obs, dtype=torch.float32, device=device).unsqueeze(0)
            act = actor.get_action(obs_t)

    next_obs, rew, terminated, truncated, _ = env.step(act)
    done = terminated or truncated

    replay.add(obs, act, rew, next_obs, done)

    obs = next_obs
    episode_return += rew
    episode_len += 1

    if done:
        obs, _ = env.reset()
        print(f"Step: {step}, Return: {episode_return:.2f}, Len: {episode_len}")
        episode_return, episode_len = 0, 0

    if step >= UPDATE_AFTER and step % UPDATE_EVERY == 0:
        for _ in range(UPDATE_EVERY):
            update(
                actor=actor,
                critic=critic,
                actor_target=actor_target,
                critic_target=critic_target,
                replay_buffer=replay,
                actor_opt=actor_opt,
                critic_opt=critic_opt,
                batch_size=BATCH_SIZE,
                gamma=GAMMA,
                tau=TAU
            )

Step: 83, Return: -299.80, Len: 84
Step: 177, Return: -63.02, Len: 94
Step: 303, Return: -320.52, Len: 126
Step: 407, Return: -278.40, Len: 104
Step: 482, Return: -111.39, Len: 75
Step: 595, Return: -366.68, Len: 113
Step: 694, Return: -94.86, Len: 99
Step: 842, Return: -227.92, Len: 148
Step: 939, Return: -262.77, Len: 97
Step: 1126, Return: -376.08, Len: 187
Step: 1224, Return: -350.42, Len: 98
Step: 1319, Return: -385.33, Len: 95
Step: 1407, Return: -69.36, Len: 88
Step: 1517, Return: 11.71, Len: 110
Step: 1611, Return: -203.74, Len: 94
Step: 1699, Return: -27.36, Len: 88
Step: 1764, Return: -65.21, Len: 65
Step: 1872, Return: -439.38, Len: 108
Step: 1978, Return: -329.29, Len: 106
Step: 2094, Return: -107.85, Len: 116
Step: 2166, Return: -53.30, Len: 72
Step: 2279, Return: -245.43, Len: 113
Step: 2416, Return: -401.22, Len: 137
Step: 2517, Return: -220.50, Len: 101
Step: 2646, Return: -255.14, Len: 129
Step: 2745, Return: -164.21, Len: 99
Step: 2863, Return: -110.72, Len: 118
Step: